In [1]:
# !conda list

In [2]:
import argparse
import glob
import logging
import os
import random
import timeit
import json
from dataclasses import dataclass, field
from typing import Dict, List, Optional

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

import transformers
from transformers import (
    HfArgumentParser,
    TrainingArguments,
    MODEL_FOR_QUESTION_ANSWERING_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    squad_convert_examples_to_features,
)
from transformers.data.metrics.squad_metrics import (
    compute_predictions_log_probs,
    compute_predictions_logits,
    squad_evaluate,
)
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor
from transformers.trainer_utils import is_main_process

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

In [3]:
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_FOR_QUESTION_ANSWERING_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [4]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

In [5]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [6]:
def train(args, train_dataset, model, tokenizer):
    model_args = args[0]
    train_args = args[1]
    """ Train the model """
    if train_args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    model_args.train_batch_size = train_args.per_gpu_train_batch_size * max(1, model_args.n_gpu)
    train_sampler = RandomSampler(train_dataset) if train_args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=model_args.train_batch_size)

    if train_args.max_steps > 0:
        t_total = train_args.max_steps
        train_args.num_train_epochs = train_args.max_steps // (len(train_dataloader) // train_args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // train_args.gradient_accumulation_steps * train_args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": train_args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=train_args.learning_rate, eps=train_args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=train_args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if os.path.isfile(os.path.join(model_args.model_name_or_path, "optimizer.pt")) and os.path.isfile(
        os.path.join(model_args.model_name_or_path, "scheduler.pt")
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(model_args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(model_args.model_name_or_path, "scheduler.pt")))

    if train_args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")

        model, optimizer = amp.initialize(model, optimizer, opt_level=train_args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if model_args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if train_args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[train_args.local_rank], output_device=train_args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", train_args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", train_args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        model_args.train_batch_size
        * train_args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if train_args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", train_args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 1
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if os.path.exists(model_args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = model_args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // train_args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // train_args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(train_args.num_train_epochs), desc="Epoch", disable=train_args.local_rank not in [-1, 0]
    )
    # Added here for reproductibility
    set_seed(train_args)

    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=train_args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            model.train()
            batch = tuple(t.to(model_args.device) for t in batch)

            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
                "start_positions": batch[3],
                "end_positions": batch[4],
            }

            if model_args.model_type in ["xlm", "roberta", "distilbert", "camembert", "bart", "longformer"]:
                del inputs["token_type_ids"]

            if model_args.model_type in ["xlnet", "xlm"]:
                inputs.update({"cls_index": batch[5], "p_mask": batch[6]})
                if model_args.version_2_with_negative:
                    inputs.update({"is_impossible": batch[7]})
                if hasattr(model, "config") and hasattr(model.config, "lang2id"):
                    inputs.update(
                        {"langs": (torch.ones(batch[0].shape, dtype=torch.int64) * args.lang_id).to(model_args.device)}
                    )

            outputs = model(**inputs)
            # model outputs are always tuple in transformers (see doc)
            loss = outputs[0]

            if model_args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel (not distributed) training
            if train_args.gradient_accumulation_steps > 1:
                loss = loss / train_args.gradient_accumulation_steps

            if train_args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % train_args.gradient_accumulation_steps == 0:
                if train_args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), train_args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), train_args.max_grad_norm)

                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                # Log metrics
                if train_args.local_rank in [-1, 0] and train_args.logging_steps > 0 and global_step % train_args.logging_steps == 0:
                    # Only evaluate when single GPU otherwise metrics may not average well
                    if train_args.local_rank == -1 and model_args.evaluate_during_training:
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / train_args.logging_steps, global_step)
                    logging_loss = tr_loss

                # Save model checkpoint
                if train_args.local_rank in [-1, 0] and train_args.save_steps > 0 and global_step % train_args.save_steps == 0:
                    output_dir = os.path.join(train_args.output_dir, "checkpoint-{}".format(global_step))
                    # Take care of distributed/parallel training
                    model_to_save = model.module if hasattr(model, "module") else model
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if train_args.max_steps > 0 and global_step > train_args.max_steps:
                epoch_iterator.close()
                break
        if train_args.max_steps > 0 and global_step > train_args.max_steps:
            train_iterator.close()
            break

    if train_args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

In [7]:
def evaluate(args, model, tokenizer, prefix=""):
    model_args = args[0]
    train_args = args[1]
    dataset, examples, features = load_and_cache_examples(args, tokenizer, evaluate=True, output_examples=True)

    if not os.path.exists(train_args.output_dir) and train_args.local_rank in [-1, 0]:
        os.makedirs(train_args.output_dir)

    model_args.eval_batch_size = train_args.per_gpu_eval_batch_size * max(1, model_args.n_gpu)

    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=model_args.eval_batch_size)

    # multi-gpu evaluate
    if model_args.n_gpu > 1 and not isinstance(model, torch.nn.DataParallel):
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(dataset))
    logger.info("  Batch size = %d", model_args.eval_batch_size)

    all_results = []
    start_time = timeit.default_timer()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(model_args.device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            if model_args.model_type in ["xlm", "roberta", "distilbert", "camembert", "bart", "longformer"]:
                del inputs["token_type_ids"]

            feature_indices = batch[3]

            # XLNet and XLM use more arguments for their predictions
            if model_args.model_type in ["xlnet", "xlm"]:
                inputs.update({"cls_index": batch[4], "p_mask": batch[5]})
                # for lang_id-sensitive xlm models
                if hasattr(model, "config") and hasattr(model.config, "lang2id"):
                    inputs.update(
                        {"langs": (torch.ones(batch[0].shape, dtype=torch.int64) * model_args.lang_id).to(model_args.device)}
                    )
            outputs = model(**inputs)

        for i, feature_index in enumerate(feature_indices):
            eval_feature = features[feature_index.item()]
            unique_id = int(eval_feature.unique_id)

            output = [to_list(output[i]) for output in outputs.to_tuple()]

            # Some models (XLNet, XLM) use 5 arguments for their predictions, while the other "simpler"
            # models only use two.
            if len(output) >= 5:
                start_logits = output[0]
                start_top_index = output[1]
                end_logits = output[2]
                end_top_index = output[3]
                cls_logits = output[4]

                result = SquadResult(
                    unique_id,
                    start_logits,
                    end_logits,
                    start_top_index=start_top_index,
                    end_top_index=end_top_index,
                    cls_logits=cls_logits,
                )

            else:
                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)

            all_results.append(result)

    evalTime = timeit.default_timer() - start_time
    logger.info("  Evaluation done in total %f secs (%f sec per example)", evalTime, evalTime / len(dataset))

    # Compute predictions
    output_prediction_file = os.path.join(train_args.output_dir, "predictions_{}.json".format(prefix))
    output_nbest_file = os.path.join(train_args.output_dir, "nbest_predictions_{}.json".format(prefix))

    if model_args.version_2_with_negative:
        output_null_log_odds_file = os.path.join(train_args.output_dir, "null_odds_{}.json".format(prefix))
    else:
        output_null_log_odds_file = None

    # XLNet and XLM use a more complex post-processing procedure
    if model_args.model_type in ["xlnet", "xlm"]:
        start_n_top = model.config.start_n_top if hasattr(model, "config") else model.module.config.start_n_top
        end_n_top = model.config.end_n_top if hasattr(model, "config") else model.module.config.end_n_top

        predictions = compute_predictions_log_probs(
            examples,
            features,
            all_results,
            model_args.n_best_size,
            model_args.max_answer_length,
            output_prediction_file,
            output_nbest_file,
            output_null_log_odds_file,
            start_n_top,
            end_n_top,
            model_args.version_2_with_negative,
            tokenizer,
            model_args.verbose_logging,
        )
    else:
        predictions = compute_predictions_logits(
            examples,
            features,
            all_results,
            model_args.n_best_size,
            model_args.max_answer_length,
            model_args.do_lower_case,
            output_prediction_file,
            output_nbest_file,
            output_null_log_odds_file,
            model_args.verbose_logging,
            model_args.version_2_with_negative,
            model_args.null_score_diff_threshold,
            tokenizer,
        )

    # Compute the F1 and exact scores.
    results = squad_evaluate(examples, predictions)
    return results

In [8]:
def load_and_cache_examples(args, tokenizer, evaluate=False, output_examples=False):
    model_args = args[0]
    train_args = args[1]
    if train_args.local_rank not in [-1, 0] and not evaluate:
        # Make sure only the first process in distributed training process the dataset, and the others will use the cache
        torch.distributed.barrier()

    # Load data features from cache or dataset file
    input_dir = model_args.data_dir if model_args.data_dir else "."
    cached_features_file = os.path.join(
        input_dir,
        "cached_{}_{}_{}".format(
            "dev" if evaluate else "train",
            list(filter(None, model_args.model_name_or_path.split("/"))).pop(),
            str(model_args.max_seq_length),
        ),
    )

    # Init features and dataset from cache if it exists
    if os.path.exists(cached_features_file) and not model_args.overwrite_cache:
        logger.info("Loading features from cached file %s", cached_features_file)
        features_and_dataset = torch.load(cached_features_file)
        features, dataset, examples = (
            features_and_dataset["features"],
            features_and_dataset["dataset"],
            features_and_dataset["examples"],
        )
    else:
        logger.info("Creating features from dataset file at %s", input_dir)

        if not model_args.data_dir and ((evaluate and not model_args.predict_file) or (not evaluate and not model_args.train_file)):
            try:
                import tensorflow_datasets as tfds
            except ImportError:
                raise ImportError("If not data_dir is specified, tensorflow_datasets needs to be installed.")

            if model_args.version_2_with_negative:
                logger.warn("tensorflow_datasets does not handle version 2 of SQuAD.")

            tfds_examples = tfds.load("squad")
            examples = SquadV1Processor().get_examples_from_dataset(tfds_examples, evaluate=evaluate)
        else:
            processor = SquadV2Processor() if model_args.version_2_with_negative else SquadV1Processor()
            if evaluate:
                examples = processor.get_dev_examples(model_args.data_dir, filename=model_args.predict_file)
            else:
                examples = processor.get_train_examples(model_args.data_dir, filename=model_args.train_file)

        features, dataset = squad_convert_examples_to_features(
            examples=examples,
            tokenizer=tokenizer,
            max_seq_length=model_args.max_seq_length,
            doc_stride=model_args.doc_stride,
            max_query_length=model_args.max_query_length,
            is_training=not evaluate,
            return_dataset="pt",
            threads=model_args.threads,
        )

        if train_args.local_rank in [-1, 0]:
            logger.info("Saving features into cached file %s", cached_features_file)
            torch.save({"features": features, "dataset": dataset, "examples": examples}, cached_features_file)

    if train_args.local_rank == 0 and not evaluate:
        # Make sure only the first process in distributed training process the dataset, and the others will use the cache
        torch.distributed.barrier()

    if output_examples:
        return dataset, examples, features
    return dataset

In [9]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """
    # Required parameters
    model_type: str = field(
        default=None,
        metadata={"help": "Model type selected in the list: " + ", ".join(MODEL_TYPES)},
    )
    model_name_or_path: str = field(
        default=None,
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"},
    )
#     output_dir: str = field(
#         default=None,
#         metadata={"help": "The output directory where the model checkpoints and predictions will be written."},
#     )
    # Other parameters
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"},
    )
    cache_dir: Optional[str] = field(
        default="", 
        metadata={"help": "Where do you want to store the pretrained models downloaded from s3"},
    )
    doc_stride: Optional[int] = field(
        default=128,
        metadata={"help":"When splitting up a long document into chunks, how much stride to take between chunks."},
    )
    max_seq_length: Optional[int] = field(
        default=384,
        metadata={"help":"The maximum total input sequence length after WordPiece tokenization. Sequences "
                  "longer than this will be truncated, and sequences shorter than this will be padded."},
    )
    max_query_length: Optional[int] = field(
        default=64,
        metadata={"help":"The maximum number of tokens for the question. Questions longer than this will "
                  "be truncated to this length."},
    )
    data_dir: Optional[str] = field(
        default=None,
        metadata={"help":"The input data dir. Should contain the .json files for the task."
                  + "If no data dir or train/predict files are specified, will run with tensorflow_datasets."},
    )
    train_file: Optional[str] = field(
        default=None,
        metadata={"help":"The input training file. If a data dir is specified, will look for the file there"
                  + "If no data dir or train/predict files are specified, will run with tensorflow_datasets."},
    )
    predict_file: Optional[str] = field(
        default=None,
        metadata={"help":"The input evaluation file. If a data dir is specified, will look for the file there"
                  + "If no data dir or train/predict files are specified, will run with tensorflow_datasets."},
    )
    config_name: Optional[str] = field(
        default="",
        metadata={"help":"Pretrained config name or path if not the same as model_name"},
    )
    version_2_with_negative: Optional[bool] = field(
        default=True,
        metadata={"help":"If true, the SQuAD examples contain some that do not have an answer."},
    )
    null_score_diff_threshold: Optional[float] = field(
        default=0.0,
        metadata={"help":"If null_score - best_non_null is greater than the threshold predict null."},
    )
#     do_train: Optional[bool] = field(
#         default=True,
#         metadata={"help":"Whether to run training."},
#     )
#     do_eval: Optional[bool] = field(
#         default=True,
#         metadata={"help":"Whether to run eval on the dev set."}
#     )
    evaluate_during_training: Optional[bool] = field(
        default=True,
        metadata={"help":"Run evaluation during training at each logging step."},
    )
    do_lower_case: Optional[bool] = field(
        default=True,
        metadata={"help":"Set this flag if you are using an uncased model."},
    )
#     per_gpu_train_batch_size: Optional[int] = field(
#         default=8, 
#         metadata={"help":"Batch size per GPU/CPU for training."},
#     )
#     per_gpu_eval_batch_size: Optional[int] = field(
#         default=8, 
#         metadata={"help":"Batch size per GPU/CPU for evaluation."},
#     )
#     learning_rate: Optional[float] = field(
#         default=5e-5, 
#         metadata={"help":"The initial learning rate for Adam."},
#     )
    
#     gradient_accumulation_steps: Optional[int] = field(
#         default=1,
#         metadata={"help":"Number of updates steps to accumulate before performing a backward/update pass."},
#     )
        
#     weight_decay: Optional[float] = field(
#         default=0.0, 
#         metadata={"help":"Weight decay if we apply some."},
#     )
        
#     adam_epsilon: Optional[float] = field(
#         default=1e-8, 
#         metadata={"help":"Epsilon for Adam optimizer."},
#     )
#     max_grad_norm: Optional[float] = field(
#         default=1.0, 
#         metadata={"help":"Max gradient norm."},
#     )
#     num_train_epochs: Optional[float] = field(
#         default=3.0, 
#         metadata={"help":"Total number of training epochs to perform."},
#     )
#     max_steps: Optional[int] = field(
#         default=-1,
#         metadata={"help":"If > 0: set total number of training steps to perform. Override num_train_epochs."},
#     )
#     warmup_steps: Optional[int] = field(
#         default=0, 
#         metadata={"help":"Linear warmup over warmup_steps."},
#     )
    n_best_size: Optional[int] = field(
        default=20,
        metadata={"help":"The total number of n-best predictions to generate in the nbest_predictions.json output file."},
    )
    max_answer_length: Optional[int] = field(
        default=30,
        metadata={"help":"The maximum length of an answer that can be generated. This is needed because the start "
                  "and end predictions are not conditioned on one another."},
    )
    verbose_logging: Optional[bool] = field(
        default=True,
        metadata={"help":"If true, all of the warnings related to data processing will be printed. "
                  "A number of warnings are expected for a normal SQuAD evaluation."},
    )
    lang_id: Optional[int] = field(
        default=0,
        metadata={"help":"language id of input for language-specific xlm models (see tokenization_xlm.PRETRAINED_INIT_CONFIGURATION)"},
    )

#     logging_steps: Optional[int] = field(
#         default=500,
#         metadata={"help":"Log every X updates steps."},
#     )
#     save_steps: Optional[int] = field(
#         default=500, 
#         metadata={"help":"Save checkpoint every X updates steps."},
#     )
    eval_all_checkpoints: Optional[bool] = field(
        default=True,
        metadata={"help":"Evaluate all checkpoints starting with the same prefix as model_name ending and ending with step number"},
    )
    n_gpu: Optional[int] = field(
        default=1,
        metadata={"help":"Run the code on number of cuda enable gpus."},
    )
#     no_cuda: Optional[bool] = field(
#         default=False,
#         metadata={"help":"Whether not to use CUDA when available"},
#     )
#     overwrite_output_dir: Optional[bool] = field(
#         default=True,
#         metadata={"help":"Overwrite the content of the output directory"},
#     )
    overwrite_cache: Optional[bool] = field(
        default=True,
        metadata={"help":"Overwrite the cached training and evaluation sets"},
    )
#     seed: Optional[int] = field(
#         default=42, 
#         metadata={"help":"random seed for initialization"},
#     )

#     local_rank: Optional[int] = field(
#         default=-1, 
#         metadata={"help":"local_rank for distributed training on gpus"},
#     )
#     fp16: Optional[bool] = field(
#         default=True,
#         metadata={"help":"Whether to use 16-bit (mixed) precision (through NVIDIA apex) instead of 32-bit"},
#     )
#     fp16_opt_level: Optional[str] = field(
#         default="O1",
#         metadata={"help":"For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']."
#                   "See details at https://nvidia.github.io/apex/amp.html"},
#     )
    
    server_ip: Optional[str] = field(
        default="", 
        metadata={"help":"Can be used for distant debugging."},
    )
    server_port: Optional[str] = field(
        default="", 
        metadata={"help":"Can be used for distant debugging."},
    )

    threads: Optional[int] = field(
        default=1, 
        metadata={"help":"multiple threads for converting example to features"},
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )
# @dataclass
# class DataTrainingArguments:
#     """
#     Arguments pertaining to what data we are going to input our model for training and eval.
#     """
#     train_file_path: Optional[str] = field(
#         default='train_data.pt',
#         metadata={"help": "Path for cached train dataset"},
#     )
#     valid_file_path: Optional[str] = field(
#         default='valid_data.pt',
#         metadata={"help": "Path for cached valid dataset"},
#     )
#     max_len: Optional[int] = field(
#         default=512,
#         metadata={"help": "Max input length for the source text"},
#     )

In [10]:
args_dict = {
    "model_type": 'bert' ,
    "model_name_or_path": 'gdario/biobert_bioasq' ,
    "do_train": True ,
    "do_eval": True ,
    "version_2_with_negative": True ,
    "train_file": '../temp_qa_scripts/train_qa.json' ,
    "predict_file": '../temp_qa_scripts/dev_qa.json' ,
    "learning_rate": 3e-5 ,
    "num_train_epochs": 4 ,
    "max_seq_length": 384 ,
    "doc_stride": 128 ,
    "output_dir": './BioBERT_squad1_cancer/' ,
    "overwrite_output_dir": True,
    "per_gpu_eval_batch_size": 20  ,
    "per_gpu_train_batch_size": 20 ,
    "save_steps": 5000,
    "n_gpu": 4,
    "do_lower_case": True,
}

with open('args.json', 'w') as f:
    json.dump(args_dict, f)

In [11]:
parser = HfArgumentParser((ModelArguments, TrainingArguments))
m, t= parser.parse_json_file(json_file=os.path.abspath('args.json'))
print(m, "\n\n", t)

ModelArguments(model_type='bert', model_name_or_path='gdario/biobert_bioasq', tokenizer_name=None, cache_dir=None, doc_stride=128, max_seq_length=384, max_query_length=64, data_dir=None, train_file='../temp_qa_scripts/train_qa.json', predict_file='../temp_qa_scripts/dev_qa.json', config_name='', version_2_with_negative=True, null_score_diff_threshold=0.0, evaluate_during_training=True, do_lower_case=True, n_best_size=20, max_answer_length=30, verbose_logging=True, lang_id=0, eval_all_checkpoints=True, n_gpu=4, overwrite_cache=True, server_ip='', server_port='', threads=1) 

 TrainingArguments(output_dir='./BioBERT_squad1_cancer/', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=20, per_gpu_eval_batch_size=20, gradient_accumulation_steps=1, eval_accumulation_steps=None, learnin

In [12]:
def main():
#     parser = argparse.ArgumentParser()
    ########## EDITED PART ###########
    parser = HfArgumentParser((ModelArguments, TrainingArguments))
    model_args, train_args = parser.parse_json_file(json_file=os.path.abspath('args.json'))
    ###################################
    #     args = parser.parse_args()
    
    print(model_args, "\n")
    print(train_args)
    
    if model_args.doc_stride >= model_args.max_seq_length - model_args.max_query_length:
        logger.warning(
            "WARNING - You've set a doc stride which may be superior to the document length in some "
            "examples. This could result in errors when building features from the examples. Please reduce the doc "
            "stride or increase the maximum length to ensure the features are correctly built."
        )

    if (
        os.path.exists(train_args.output_dir)
        and os.listdir(train_args.output_dir)
        and train_args.do_train
        and not train_args.overwrite_output_dir
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                train_args.output_dir
            )
        )

    # Setup distant debugging if needed
    if model_args.server_ip and model_args.server_port:
        # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
        import ptvsd

        print("Waiting for debugger attach")
        ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
        ptvsd.wait_for_attach()

    # Setup CUDA, GPU & distributed training
    if train_args.local_rank == -1 or train_args.no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not train_args.no_cuda else "cpu")
        model_args.n_gpu = 0 if train_args.no_cuda else torch.cuda.device_count()
    else:  # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.cuda.set_device(train_args.local_rank)
        device = torch.device("cuda", train_args.local_rank)
        torch.distributed.init_process_group(backend="nccl")
        model_args.n_gpu = 1
    model_args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if train_args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        train_args.local_rank,
        device,
        model_args.n_gpu,
        bool(train_args.local_rank != -1),
        train_args.fp16,
    )

    # Set the verbosity to info of the Transformers logger (on main process only):
    if is_main_process(train_args.local_rank):
        transformers.utils.logging.set_verbosity_info()
        transformers.utils.logging.enable_default_handler()
        transformers.utils.logging.enable_explicit_format()
        
    # Set seed
    set_seed(train_args)
#     set_seed(model_args)
    
    # Load pretrained model and tokenizer
    if train_args.local_rank not in [-1, 0]:
        # Make sure only the first process in distributed training will download model & vocab
        torch.distributed.barrier()

    model_args.model_type = model_args.model_type.lower()
    config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir if model_args.cache_dir else None,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        do_lower_case=model_args.do_lower_case,
        cache_dir=model_args.cache_dir if model_args.cache_dir else None,
        use_fast=False,  # SquadDataset is not compatible with Fast tokenizers which have a smarter overflow handeling
    )
    model = AutoModelForQuestionAnswering.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir if model_args.cache_dir else None,
    )

    if train_args.local_rank == 0:
        # Make sure only the first process in distributed training will download model & vocab
        torch.distributed.barrier()

    model.to(model_args.device)

    logger.info("Training/evaluation parameters %s", train_args)

    # Before we do anything with models, we want to ensure that we get fp16 execution of torch.einsum if args.fp16 is set.
    # Otherwise it'll default to "promote" mode, and we'll get fp32 operations. Note that running `--fp16_opt_level="O2"` will
    # remove the need for this code, but it is still valid.
    if train_args.fp16:
        try:
            import apex

            apex.amp.register_half_function(torch, "einsum")
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")

    # Training
    if train_args.do_train:
        train_dataset = load_and_cache_examples((model_args, train_args), tokenizer, evaluate=False, output_examples=False)
        global_step, tr_loss = train((model_args, train_args), train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Save the trained model and the tokenizer
    if train_args.do_train and (train_args.local_rank == -1 or torch.distributed.get_rank() == 0):
        logger.info("Saving model checkpoint to %s", train_args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        # Take care of distributed/parallel training
        model_to_save = model.module if hasattr(model, "module") else model
        model_to_save.save_pretrained(train_args.output_dir)
        tokenizer.save_pretrained(train_args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save((model_args, train_args), os.path.join(train_args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelForQuestionAnswering.from_pretrained(train_args.output_dir)  # , force_download=True)
        
        # SquadDataset is not compatible with Fast tokenizers which have a smarter overflow handeling
        # So we use use_fast=False here for now until Fast-tokenizer-compatible-examples are out
        tokenizer = AutoTokenizer.from_pretrained(train_args.output_dir, do_lower_case=model_args.do_lower_case, use_fast=False)
        model.to(model_args.device)

    # Evaluation - we can ask to evaluate all the checkpoints (sub-directories) in a directory
    results = {}
    if train_args.do_eval and train_args.local_rank in [-1, 0]:
        if train_args.do_train:
            logger.info("Loading checkpoints saved during training for evaluation")
            checkpoints = [train_args.output_dir]
            if model_args.eval_all_checkpoints:
                checkpoints = list(
                    os.path.dirname(c)
                    for c in sorted(glob.glob(train_args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
                )

        else:
            logger.info("Loading checkpoint %s for evaluation", model_args.model_name_or_path)
            checkpoints = [model_args.model_name_or_path]

        logger.info("Evaluate the following checkpoints: %s", checkpoints)

        for checkpoint in checkpoints:
            # Reload the model
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)  # , force_download=True)
            model.to(model_args.device)

            # Evaluate
            result = evaluate((model_args, train_args), model, tokenizer, prefix=global_step)

            result = dict((k + ("_{}".format(global_step) if global_step else ""), v) for k, v in result.items())
            results.update(result)

    logger.info("Results: {}".format(results))

    return results

In [13]:
main()

11/25/2020 16:36:58 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 4, distributed training: False, 16-bits training: False


ModelArguments(model_type='bert', model_name_or_path='gdario/biobert_bioasq', tokenizer_name=None, cache_dir=None, doc_stride=128, max_seq_length=384, max_query_length=64, data_dir=None, train_file='../temp_qa_scripts/train_qa.json', predict_file='../temp_qa_scripts/dev_qa.json', config_name='', version_2_with_negative=True, null_score_diff_threshold=0.0, evaluate_during_training=True, do_lower_case=True, n_best_size=20, max_answer_length=30, verbose_logging=True, lang_id=0, eval_all_checkpoints=True, n_gpu=4, overwrite_cache=True, server_ip='', server_port='', threads=1) 

TrainingArguments(output_dir='./BioBERT_squad1_cancer/', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=8, per_device_eval_batch_size=8, per_gpu_train_batch_size=20, per_gpu_eval_batch_size=20, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning

[INFO|configuration_utils.py:414] 2020-11-25 16:36:58,800 >> loading configuration file https://huggingface.co/gdario/biobert_bioasq/resolve/main/config.json from cache at /home/tr27p/.cache/huggingface/transformers/7e8b73270ffbea99d89d1a733d62420fcee4188934d4a3bcf8ba6448b627ad43.57ec8f6d1bdd1dc2c389b6ac7edf7fcd55f5fb5cd8969a43e1a882eba35d7ee2
[INFO|configuration_utils.py:450] 2020-11-25 16:36:58,801 >> Model config BertConfig {
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "type_vocab_size": 2,
  "vocab_size": 28996
}

[INFO|configuration_utils.p

Iteration:  83%|████████▎ | 30/36 [00:27<00:05,  1.10it/s]


Epoch: 100%|██████████| 4/4 [02:23<00:00, 35.80s/it]
11/25/2020 16:40:23 - INFO - __main__ -    global_step = 145, average loss = 0.06826759999445318
11/25/2020 16:40:23 - INFO - __main__ -   Saving model checkpoint to ./BioBERT_squad1_cancer/
[INFO|configuration_utils.py:284] 2020-11-25 16:40:23,767 >> Configuration saved in ./BioBERT_squad1_cancer/config.json
[INFO|modeling_utils.py:741] 2020-11-25 16:40:24,455 >> Model weights saved in ./BioBERT_squad1_cancer/pytorch_model.bin
[INFO|configuration_utils.py:412] 2020-11-25 16:40:24,479 >> loading configuration file ./BioBERT_squad1_cancer/config.json
[INFO|configuration_utils.py:450] 2020-11-25 16:40:24,480 >> Model config BertConfig {
  "_name_or_path": "gdario/biobert_bioasq",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3

{'exact': 79.45205479452055,
 'f1': 81.62469527719749,
 'total': 73,
 'HasAns_exact': 79.45205479452055,
 'HasAns_f1': 81.62469527719749,
 'HasAns_total': 73,
 'best_exact': 79.45205479452055,
 'best_exact_thresh': 0.0,
 'best_f1': 81.62469527719749,
 'best_f1_thresh': 0.0}